In [43]:
import pandas as pd
import numpy as np
import tweepy
from sklearn.ensemble import RandomForestClassifier

### targets:
####    0: porn
####    1: propaganda
####    2: spam
####    3: fake followers
####    4: genuine accounts

# Test users
### 3201005265 - porn - 0
### 3593771535 - porn - 0
### 126523026 - spam - 2
### 3318703632 - propaganda - 1
### 875124919 - propaganda - 1

In [46]:
# create connection with Twitter API

CONSUMER_KEY = 'J3uhwWHdmSt3uD69ry8r2kc3B'
CONSUMER_SECRET = 'LeuM7dVfFbXH0bc5fFkZIxlNiUXyjIx4Kcjh58HeWs2TrLijDx'
ACCESS_TOKEN = '327497511-5dNjiGaTQHASljoxNr1qlMOlHrrSB21HBFBDjx2E'
ACCESS_TOKEN_SECRET = 'tHLqwlhHYJWptW3femyq0rbMW6ZItu5yLQ3DFGJqJ8Xlg'

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

In [47]:
ids = [3201005265, 3593771535, 126523026, 3318703632, 875124919]

In [48]:
test_set = []
for elem in ids:
    test_set.append(api.get_user(elem))

In [49]:
df = pd.DataFrame(columns=["statuses_count","followers_count","friends_count","favourites_count","listed_count","url","default_profile_image","profile_use_background_image","profile_background_image_url_https","profile_background_tile","profile_background_image_url","description"])
for user in test_set:
    df2 = pd.DataFrame([[user.statuses_count,
                            user.followers_count,
                            user.friends_count,
                            user.favourites_count,
                            user.listed_count,
                            user.url,
                            user.default_profile_image,
                            user.profile_use_background_image,
                            user.profile_background_image_url_https,
                            user.profile_background_tile,
                            user.profile_background_image_url,
                            user.description]],
                            columns=["statuses_count","followers_count","friends_count","favourites_count","listed_count","url","default_profile_image","profile_use_background_image","profile_background_image_url_https","profile_background_tile","profile_background_image_url","description"]
                          )
    df = df.append(df2, ignore_index=True)

In [50]:
df['description'] = df['description'].fillna('')
df['description'] = df['description'].apply(lambda x: len(x))

In [64]:
df['url']

0                      None
1                      None
2    http://t.co/zCj321TiF2
3                      None
4                      None
Name: url, dtype: object

In [54]:
df['default_profile_image'] = df['default_profile_image'].astype(int)

In [56]:
df['profile_background_tile'] = df['profile_background_tile'].astype(int)

In [58]:
df['profile_use_background_image'] = df['profile_use_background_image'].astype(int)

In [61]:
df['profile_background_image_url'] = df['profile_background_image_url'].apply(lambda x: int(not(int(pd.isnull(x)))))

In [63]:
df['profile_background_image_url_https'] = df['profile_background_image_url_https'].apply(lambda x: int(not(int(pd.isnull(x)))))

In [65]:
df['url'] = df['url'].apply(lambda x: int(not(int(pd.isnull(x)))))

In [66]:
df

,statuses_count,followers_count,friends_count,favourites_count,listed_count,url,default_profile_image,profile_use_background_image,profile_background_image_url_https,profile_background_tile,profile_background_image_url,description
0,304192,6495,6883,258805,301,0,0,0,1,0,1,160
1,253,376,1192,0,1,0,0,1,1,0,1,0
2,41766,17,0,0,1,1,0,1,1,0,1,16
3,2743,100,837,1499,1,0,0,1,1,1,1,0
4,32277,44,38,0,12,0,1,1,1,0,1,0


In [67]:
train_set = pd.read_csv('data/full/baseline_dataset.csv', encoding='utf-8-sig')

In [78]:
train_set = train_set.drop('Unnamed: 0', axis=1)

In [83]:
df = df.reindex_axis(sorted(df.columns), axis=1)

/Users/Lorenzo/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  if __name__ == '__main__':


In [69]:
df.to_csv('data/full/baseline_testset.csv')

In [79]:
target = 'target'
features = train_set.columns[train_set.columns!=target]

In [80]:
X = train_set[features]
y = train_set[target]

In [81]:
# Fitting Random Forest Classification to the Training set
forest = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 42)
forest.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [84]:
df

,default_profile_image,description,favourites_count,followers_count,friends_count,listed_count,profile_background_image_url,profile_background_image_url_https,profile_background_tile,profile_use_background_image,statuses_count,url
0,0,160,258805,6495,6883,301,1,1,0,0,304192,0
1,0,0,0,376,1192,1,1,1,0,1,253,0
2,0,16,0,17,0,1,1,1,0,1,41766,1
3,0,0,1499,100,837,1,1,1,1,1,2743,0
4,1,0,0,44,38,12,1,1,0,1,32277,0


In [85]:
train_set

,default_profile_image,description,favourites_count,followers_count,friends_count,listed_count,profile_background_image_url,profile_background_image_url_https,profile_background_tile,profile_use_background_image,statuses_count,target,url
0,1,54,381,15,185,0,1,1,1,1,80,0,0
1,1,38,43,29,43,1,1,1,1,1,11,0,0
2,1,0,56,7,74,0,1,1,1,1,10,0,0
3,1,40,491,217,369,0,1,1,1,1,7372,0,0
4,1,64,241,10,17,0,1,1,1,1,55,0,0
5,1,44,141,9,130,0,1,1,1,1,31,0,0
6,1,83,0,297,4801,0,1,1,1,1,1,0,1
7,1,0,116,8,202,0,1,1,1,1,23,0,0
8,1,38,180,5,31,0,1,1,1,1,56,0,0
9,1,52,399,19,52,0,1,1,1,1,69,0,0
